# PART 3: Connect with Manufacturers for remote SMC

In [1]:
%%javascript
document.title="City Agent"

<IPython.core.display.Javascript object>

#### Imports

In [2]:
%autoawait 
from aries_cloudcontroller import AriesAgentController
#import asyncio
import os
from pprintpp import pprint
from termcolor import colored
import time
import json
import libs.functions as fnc

IPython autoawait is `on`, and set to use `asyncio`


In [3]:
# If the identifiers are not stored for some reason
try:
    # Load variables – assuming they have been written to the store in a previous notebook
    print("--- LOAD VARIABLES FROM JUPYTERLAB STORE ---")
    
    %store -r schema_manufacturer_id
    %store -r cred_def_manufacturer_id
    %store -r authority_did
    
    print(f"schema_manufacturer_id: {schema_manufacturer_id}")
    print(f"cred_def_manufacturer_id: {cred_def_manufacturer_id}")
    print(f"authority_did: {authority_did}")
    
except Exception as e:
    
    print("--- DEFINE VARIABLES FROM COPIED identifiers DICT ---")
    
    # Copy an updated version if necessary
    identifiers = get_identifiers()

    # Get manufacturer schema identifiers
    schema_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]
    cred_def_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["cred_def"]
    
    # Get authority did
    authority_did = identifiers['authority_did']

    # Load variables to Jupyter store
    %store schema_manufacturer_id
    %store cred_def_manufacturer_id
    %store authority_did

--- LOAD VARIABLES FROM JUPYTERLAB STORE ---
schema_manufacturer_id: MTXSc4YD8wixyM9ekZbmMC:2:certify-manufacturer:0.0.1
cred_def_manufacturer_id: MTXSc4YD8wixyM9ekZbmMC:3:CL:246687:default
authority_did: MTXSc4YD8wixyM9ekZbmMC


#### Initialize `City` Agent Controller

In [4]:
# Get relevant attributes
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

## 1 – Init Webhook Server and register Event Listeners for it
#### 1.1 – Start webhook server
Start a webhook server to be able to communicate with other agents

In [5]:
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Initialising a controller with admin api at http://city-agent:3021 and an api key of adminApiKey
Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.2 – Define listeners
Define listeners that are triggered when something happens on the webhook server

In [6]:
# Init listener list
listeners = []

# Add listeners defined in previous cell
connection_listener = {"handler": fnc.connections_handler, "topic": "connections"}
listeners.append(connection_listener)

verifier_listener = {"topic": "present_proof", "handler": fnc.verifier_proof_handler}
listeners.append(verifier_listener)

# Register listeners
agent_controller.register_listeners(listeners)

## Define Presentation Request Object

The below cell defines a generic presentation request object, that can be sent across specific connections requesting that they produce a presentation containing the identified attributes and meeting the restrictions.

It is often useful to define your request objects first, then reuse these objects across many connections you wish to request a proof from. 

Duplicate and customise the below cell as many times as you need. It may be useful to save these request objects either to the jupyter store using %store or through

TODO: Detail the full set of restrictions available to a verifier.

In [7]:
# List of required attribuets and restrictions
req_attrs = [
    {"name": "isManufacturer", "restrictions": [{"schema_id": schema_manufacturer_id}]},
]

# Define proof_requeset
manufacturer_proof_request = {
    "name": "isManufacturer Proof Request",
    "version": "1.0",
    "requested_attributes": {f"0_{req_attr['name']}_uuid": req_attr for req_attr in req_attrs },
    # Ignore predicates (e.g., range proofs)
    "requested_predicates": {}, #{f"0_{req_pred['name']}_GE_uuid": req_pred for req_pred in req_preds },
    # You can also request the entire proof request be non-revoked
    "non_revoked":  {"to": int(time.time())}
}

In [8]:
manufacturer_proof_request

{'name': 'isManufacturer Proof Request',
 'version': '1.0',
 'requested_attributes': {'0_isManufacturer_uuid': {'name': 'isManufacturer',
   'restrictions': [{'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-manufacturer:0.0.1'}]}},
 'requested_predicates': {},
 'non_revoked': {'to': 1628614923}}

## Create Invitation
Note the current arguments specified are in their default configurations. 

In [9]:
alias = None # Alias for invited connection
auto_accept = "true" # Auto accept response
public = "false" # Don't use a public DID
multi_use = "true" # Use invitation for multiple invitees

await fnc.create_invitation(agent_controller, alias, auto_accept, public, multi_use)


----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  92fe868e-2e8d-4241-aa8d-4a14857e4d7e
State : invitation
Routing State : none (invitation-sent)
Their Role :  invitee
----------------------------------------------------------

Copy & paste invitation and share with external agent:
{
    '@id': 'e279caf3-433a-4707-bc57-65a42c9cfd94',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'City',
    'recipientKeys': ['5h9MaLbBUhs16dK3N4rR5bqNKCK4RXDLDsMeZ81RxDAU'],
    'serviceEndpoint': 'https://45cdcbf290ad.ngrok.io',
}




## Share Invitation Object with External Agent

Typically in this jupyter notebook playground that involves copying it across to another agent's business logic notebook where they are the invitee. (see prover_template)

In [ ]:
#invitation = invite["invitation"]
#pprint(invitation) 

## Optional: Recieve Proof Proposal

A Verifier is not the only role that can initiate the present-proof protocol. A holder can also send a proposal for a presentation that they can provide.

In [ ]:
# TODO...

## Send Proof Request

This identifies a connection and a proof request object (you will need to update the variable name enclosed with <>) 

In [ ]:
# Step 1

proof_request = {
    "comment": "Please prove that you are an agent who is a certified manufacturer",
    "connection_id": fnc.CONNECTION_ID,
    "proof_request": manufacturer_proof_request,
    "trace": False
}

proof_request_response = await agent_controller.proofs.send_request(proof_request)

*I THINK* **Break point**: see step 2

---

## Get Presentation Exchange Record

This record keeps track of the current state of the presentation protocol, which must be in the `presentation_received` state before the presentation can be verified.

Note: This could also happen in the webhook logic.

In [ ]:
presentation_exchange_id = proof_request_response["presentation_exchange_id"]
pres_record = await agent_controller.proofs.get_record_by_id(presentation_exchange_id)
print(pres_record)

## Verify Presentation

Only if it is in the right state. 

Note: Verifying a presentation moves the state to `verified` regardless of whether the presentation request has been satisfied. To check this you must refer to the `verified` property on the response.

In [ ]:
verified_response = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
verified = verified_response["verified"]

## Parsing Disclosed Attribute Values from Presentation

A presentation object contains three classes of attributes. 
* Revealed Attributes: Attributes that were signed by an issuer and have been revealed in the presentation process
* Self Attested Attributes: Attributes that the prover has self attested to in the presentation object.
* Predicate proofs: Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

### Parse Revealed Attributes

In [ ]:
for (name, val) in verified_response['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print("\nAttribute : ", val)
    
    attr_name = verified_response["presentation_request"]["requested_attributes"][name]["name"]
    print("Attribute Name :  Raw Value")
    print(f"{attr_name}   :  {val['raw']}")

### Parse Self-Attested Attributes

In [ ]:
for (name, val) in verified_response['presentation']['requested_proof']['self_attested_attrs'].items():
    print(name)
    ## Slightly different for self attested attrs
    print(val)

## Your Own Business Logic

Now you should have an established, active connection you can write any custom logic you want to engage with protocols with the connection

In [1]:
# Imports
import torch
import syft as sy

#from sympc.session import Session
#from sympc.session import SessionManager
#from sympc.tensor import MPCTensor

In [2]:
# Step 4: Connect to duet of M1
duet_m1 = sy.duet("85d48ff60939a26bf519c5c11a796d6c")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 2d2bb6d4beff85f3b77fc1ed89b2a3c2

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


In [ ]:
print(f"duet id: {duet_m1.id}, duet target it: {duet_m1.target_id}, duet note: {duet_m1.node}")

In [3]:
duet_m1.store.pandas

,ID,Tags,Description,object_type
0,<UID: c44a05b21c2e4bc1b0f7e0dd18b0100a>,[test_data_age],description for age_data,<class 'torch.Tensor'>


In [ ]:
from IPython.core.debugger import set_trace

duet_m1.store.pandas

## 2 - Secure Multi-Party Computation

In [ ]:
# Step 5: Setup session with all duets
session = Session(parties=[duet_m1, duet_ds])
print(session)

In [ ]:
# Step 6: Setup MPC session
SessionManager.setup_mpc(session)

In [ ]:
duet_ds.store.pandas

In [ ]:
from IPython.core.debugger import set_trace

# Step 7: NOW we're ready to set up private operaitions
%pdb
duet_m1.store.pandas

In [ ]:
# Step 12: basic operations
x_secret = duet_m1.store["TEST_#1_from_m1_to_ds"] # describe local data to test sum, substract, and multiply

In [ ]:
y = torch.Tensor([[-5,8,1,7,6,100]]) # Local data

In [ ]:
x = MPCTensor(secret=x_secret, shape=(1,), session=session)  # MPC Tensor from x_secret

The following lines of codes always receive an error message.
[See Github issue](https://github.com/OpenMined/SyMPC/issues/282).

In [ ]:
print("X + Y = ",(x + y).reconstruct())

In [ ]:
print("X - Y = ", (x - y).reconstruct())

In [ ]:
print("X * Y = ", (x * y).reconstruct())

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()